In [1]:
import nltk
import re
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
news = pd.read_csv('news_for_classification_final.csv')
news

news       date                                              title  \
0        news1   2-Jan-19  germany's mothballed wilhelmshaven to restart ...   
1        news2   2-Jan-19  italy isab successfully ends 53-day maintenanc...   
2        news3   2-Jan-19  italian falconara's planned jan 1 works delaye...   
3        news4   2-Jan-19  poland's gdansk operator sees q4 margin rise 1...   
4        news5   2-Jan-19  ioc hopeful of iranian funding for cpcl refine...   
...        ...        ...                                                ...   
1316  news1317  30-Dec-19  tecnicas, samsung awarded algerian project for...   
1317  news1318  30-Dec-19  italy's sarroch experiences leak at ship-loadi...   
1318  news1319  30-Dec-19  china's zhongjie petrochemical exports first g...   
1319  news1320  31-Dec-19  uzbekneftegaz starts project to increase refin...   
1320  news1321  31-Dec-19  italy's sarroch completes planned q4 works: so...   

      label                                              notes  
0         1  notes: hestya energy aims to start operations ...  
1         1  notes: the isab refinery in southern italy has...  
2         1  notes: the api refinery in the central italian...  
3         0  notes: poland's second-largest refiner, grupa ...  
4         0  notes: state-owned indian oil corp is hopeful ...  
...     ...                                                ...  
1316      0  notes: spanish engineering group tecnicas reun...  
1317      0  notes: italy's sarroch refinery experienced a ...  
1318      0  notes: the 24,000 b/d cnooc oil zhongjie petro...  
1319      0  notes: uzbekneftegaz along with ernst&young is...  
1320      1  notes: italy's sarroch refinery on the island ...  

[1321 rows x 5 columns]

In [25]:
# create table from txt files
# def create_table():
#     # l is the table and every item of l is the row of the table
#     l = []
#     for i in range(1,1322):
        
#         # temp is the row of table and every item is the cell of table
#         temp = []
#         fdir='news/news'+str(i)+'.txt'
#         with open(fdir,'r') as file:
#             news = file.read()

#             # words tokenization
#             news_words = nltk.word_tokenize(news)
            
#             # preserve english words
#             news_al = [w for w in news_words if re.search('^[a-z]+$', w)]
            
#             # remove stop words
#             news_nonsw = [w for w in news_al if w not in stop_list]
            
#             # stemmer
#             news_stem = [stemmer.stem(w) for w in news_nonsw]
            
#             temp.append('news'+str(i))
#             temp.append(news_stem)
#             l.append(temp)
            
#     # initialize data frame
#     document = pd.DataFrame(l,columns=['news','words'])
#     document['count'] = document['words'].apply(lambda x: Counter(x))
    
#     return document

In [3]:
# create words table from csv file
def create_table():

    news = pd.read_csv('news_for_classification_final.csv')
    news['text'] = news['title'] + news['notes']
    
    # words tokenization
    news['words'] = news['text'].apply(lambda x: nltk.word_tokenize(x))
    
    # preserve english words
    news['char'] = news['words'].apply(lambda x: [w for w in x if re.search('^[a-z]+$', w)])
    
    # remove stop words
    news['nonsw'] = news['char'].apply(lambda x: [w for w in x if w not in stop_list])    
            
    # stemmer
    news['stem'] = news['nonsw'].apply(lambda x: [stemmer.stem(w) for w in x])
            
    # initialize data frame
    document = news[['news','date','title','label','notes','stem']].copy()
    document['count'] = document['stem'].apply(lambda x: Counter(x))
    
    return document

In [4]:
# get word frequency
def count_words(document, key_words):
    
    # initialize the total column
    document['total'] = 0
    
    # count every key words
    for i in range(len(key_words)):
        w = stemmer.stem(key_words[i])
        document[key_words[i]] = document['count'].apply(lambda x: x[w] if w in x.keys() else 0)
        
        # add to total column
        document['total'] += document[key_words[i]]
        
    return document

In [5]:
# get the summary of the results
def summary(document, key_words, n):
    
    # print total
    print('total:')
    news_no = document.nlargest(n, 'total')['news']
    print(news_no)
    print()
    
    # print seperate
    for i in range(len(key_words)):
        print(key_words[i]+':')
        news_no = document.nlargest(n, key_words[i])['news']
        print(news_no)
        print()

In [6]:
# print out the details of the results
def print_out(document, key_words, n):
    
    # print total
    print('total:')
    news_no = document.nlargest(n, 'total')['news']
    for i in range(n):
        news_detail = document[document['news'] == news_no.iloc[i]]
        print('\nresult ',i+1,' words count: ', news_detail.values.tolist()[0][7])
        print('news number: ', news_detail.values.tolist()[0][0])
        print('news date: ', news_detail.values.tolist()[0][1])
        print('news title: ', news_detail.values.tolist()[0][2])
        print('news label: ', news_detail.values.tolist()[0][3])
        print('news notes: ', news_detail.values.tolist()[0][4])
    print()
    
    # print seperate
    for i in range(len(key_words)):
        print(key_words[i]+':')
        news_no = document.nlargest(n, key_words[i])['news']
        for j in range(n):
            news_detail = document[document['news'] == news_no.iloc[j]]
            print('\nresult ',j+1,' words count: ', news_detail.values.tolist()[0][7])
            print('news number: ', news_detail.values.tolist()[0][0])
            print('news date: ', news_detail.values.tolist()[0][1])
            print('news title: ', news_detail.values.tolist()[0][2])
            print('news label: ', news_detail.values.tolist()[0][3])
            print('news notes: ', news_detail.values.tolist()[0][4])
        print()

In [7]:
# main part

# determine the key words, use ',' to split them
# key_words = ['maintenance']

# soppuse we want to retrieve the documents about 'maintenance,shut,restart' which may influence the oil production
key_words = input().split(',')
key_words

maintenance,shut,restart


['maintenance', 'shut', 'restart']

In [8]:
# get stop words and stemmer
stop_list = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.porter.PorterStemmer()

# create words table
document = create_table()
# get word frequency
document = count_words(document, key_words)
# visualize
document.head()

news      date                                              title  label  \
0  news1  2-Jan-19  germany's mothballed wilhelmshaven to restart ...      1   
1  news2  2-Jan-19  italy isab successfully ends 53-day maintenanc...      1   
2  news3  2-Jan-19  italian falconara's planned jan 1 works delaye...      1   
3  news4  2-Jan-19  poland's gdansk operator sees q4 margin rise 1...      0   
4  news5  2-Jan-19  ioc hopeful of iranian funding for cpcl refine...      0   

                                               notes  \
0  notes: hestya energy aims to start operations ...   
1  notes: the isab refinery in southern italy has...   
2  notes: the api refinery in the central italian...   
3  notes: poland's second-largest refiner, grupa ...   
4  notes: state-owned indian oil corp is hopeful ...   

                                                stem  \
0  [germani, mothbal, wilhelmshaven, restart, uni...   
1  [itali, isab, success, end, mainten, sourc, is...   
2  [italian, falconara, plan, jan, work, delay, s...   
3  [poland, gdansk, oper, see, margin, rise, quar...   
4  [ioc, hope, iranian, fund, cpcl, refineri, exp...   

                                               count  total  maintenance  \
0  {'germani': 2, 'mothbal': 1, 'wilhelmshaven': ...      2            0   
1  {'itali': 2, 'isab': 2, 'success': 2, 'end': 1...      3            3   
2  {'italian': 2, 'falconara': 2, 'plan': 2, 'jan...      2            2   
3  {'poland': 2, 'gdansk': 1, 'oper': 1, 'see': 1...      0            0   
4  {'ioc': 3, 'hope': 2, 'iranian': 2, 'fund': 1,...      0            0   

   shut  restart  
0     0        2  
1     0        0  
2     0        0  
3     0        0  
4     0        0

In [9]:
# get summary of the results: top n document number which has the highest tf of certain word or word list
summary(document, key_words, 5)

total:
643    news644
482    news483
719    news720
732    news733
199    news200
Name: news, dtype: object

maintenance:
719    news720
78      news79
84      news85
199    news200
206    news207
Name: news, dtype: object

shut:
482    news483
229    news230
266    news267
300    news301
732    news733
Name: news, dtype: object

restart:
331    news332
662    news663
198    news199
224    news225
611    news612
Name: news, dtype: object



In [9]:
# get details of the results: help users locate the most relevant documents
# we can see that most of the labels are 1, the maintenance,shut,restart has a highly possibility of decrease the oil production
print_out(document, key_words, 5)

total:

result  1  words count:  9
news number:  news644
news date:  27-Jun-19
news title:  china's lanzhou, urumqi, jinling, luoyang back from work, works at harbin .
news label:  1
news notes:  notes: several refineries in china that had been carrying out works are back
from maintenance, refinery sources said. those include petrochina's lanzhou petrochemical company in northwestern gansu province, which shut for a scheduled full maintenance from end-april until june 18.     separately, the urumqi refinery, also owned by petrochina, is back after being shut for a full turnaround over april 21-june 19.     works have also been completed at sinopec's jinling petrochemical which has restarted the 8 million mt/year cdu and two secondary units after completing a two-month maintenance around june 15.     sinopec's luoyang petrochemical refinery has restarted from maintenance from around june 18 after shutting down at around april 20.      meanwhile, works have started at the harbin petroche